In [2]:
from glob import glob
import pandas as pd
import xarray as xr

import contextlib
import os
import pathlib
import re
import shutil
import time
from datetime import datetime
from typing import Union

import joblib
import numpy as np
import pandas as pd
import xarray as xr
from joblib import Parallel, delayed
from tqdm import tqdm

from pandas.testing import assert_frame_equal

from pismragis.processing import convert_netcdf_to_dataframe, ncfile2dataframe
from pismragis.analysis import prepare_df, sensitivity_analysis


In [6]:
infiles = glob("../tests/data/ts_gris_g1200m_v2023_RAGIS_id_*_1980-1-1_2020-1-1.nc")
df = convert_netcdf_to_dataframe(infiles, add_vars=False)
df.to_parquet("../tests/data/test_scalar.parquet")
df.to_csv("../tests/data/test_scalar.csv")

Converting netcdf files to pandas.DataFrame
-------------------------------------------


Processing files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 110.34it/s]


Conversion finished in 0 seconds
-------------------------------------------


In [33]:
df = convert_netcdf_to_dataframe(infiles, add_vars=False, resample="yearly")
df = df[df.time.between("1980-01-01", "1982-01-01")].reset_index(drop=True)
df.to_parquet("../tests/data/test_scalar_YM.parquet")
df.to_csv("../tests/data/test_scalar_YM.csv")


Converting netcdf files to pandas.DataFrame
-------------------------------------------


Processing files: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:01<00:00, 23.99it/s]

Conversion finished in 1 seconds
-------------------------------------------


In [13]:
infile = "../tests/data/ts_gris_g1200m_v2023_RAGIS_id_0_1980-1-1_2020-1-1.nc"

In [14]:
df = ncfile2dataframe(infile, add_vars=False)
df.to_parquet("../tests/data/test_scalar_file.parquet")
df.to_csv("../tests/data/test_scalar_file.csv")

In [15]:
df = ncfile2dataframe(infile, add_vars=True)
df.to_parquet("../tests/data/test_scalar_file_add_vars.parquet")
df.to_csv("../tests/data/test_scalar_file_add_vars.csv")

In [39]:
n_jobs = 1

In [63]:
    X_df = (
        pd.read_parquet("../tests/data/test_scalar_YM.parquet")
        .drop(columns=["Year", "resolution_m"])
        .sort_values(by=["time", "id"])
    )
    ensemble_file = "../tests/data/gris_ragis_ocean_simple_lhs_50_w_posterior.csv"
    Y_df = sensitivity_analysis(
        X_df, ensemble_file=ensemble_file, n_jobs=n_jobs, seed=seed
    )
    Y_df.to_parquet("../tests/data/test_sensitivity.parquet")


-------------------------------------------

Program finished in 4 seconds


In [64]:
    Y_true_df = pd.read_parquet("../tests/data/test_sensitivity.parquet")
    Y_df = sensitivity_analysis(
        X_df, ensemble_file=ensemble_file, n_jobs=n_jobs, seed=seed
    )
    assert_frame_equal(Y_df, Y_true_df, rtol=1e-04)


-------------------------------------------

Program finished in 4 seconds


In [71]:
    ds = xr.open_mfdataset(
        "../tests/data/ts_gris_g1200m_v2023_RAGIS_id_*.nc",
        combine="nested",
        concat_dim="id",
        preprocess=nc_add_id,
        parallel=True,
    )
    ens = (
        ds.sel(time=slice("1980-01-01", "1982-01-01"))[ens_vars_dict.keys()]
        .resample(time="1AS")
        .mean()
    )
    X_xr_df = (
        ens.to_dataframe()
        .rename(columns=ens_vars_dict)
        .reset_index()
        .dropna()
        .sort_values(by=["time", "id"])
        .reset_index(drop=True)
    )

    Y_xr_df = sensitivity_analysis(
        X_xr_df, ensemble_file=ensemble_file, n_jobs=n_jobs, seed=seed
    )
    assert_frame_equal(Y_xr_df, Y_true_df, atol=1e-01, rtol=1e-06)


-------------------------------------------

Program finished in 4 seconds


In [5]:
    infile = "../tests/data/ts_gris_g1200m_v2023_RAGIS_id_0_1980-1-1_2020-1-1.nc"

    df_parquet_true = pd.read_parquet("../tests/data/test_scalar_file.parquet")
    df_csv_true = pd.read_csv(
        "../tests/data/test_scalar_file.csv",
        index_col=0,
        infer_datetime_format=True,
        parse_dates=["time"],
    )

    df = ncfile2dataframe(infile, add_vars=False)


In [8]:
    assert_frame_equal(df, df_parquet_true)


In [7]:
df_parquet_true

,time,id,grounding_line_flux (Gt year-1),limnsw (kg),Year,resolution_m
0,1980-01-02,0,-549.268836,2.633908e+18,1980.002732,1200
1,1980-01-03,0,-531.800248,2.633909e+18,1980.005464,1200
2,1980-01-04,0,-524.580911,2.633910e+18,1980.008197,1200
3,1980-01-05,0,-542.280141,2.633911e+18,1980.010929,1200
4,1980-01-06,0,-582.518599,2.633911e+18,1980.013661,1200
...,...,...,...,...,...,...
14605,2019-12-28,0,-434.405629,2.632364e+18,2019.989041,1200
14606,2019-12-29,0,-430.935017,2.632365e+18,2019.991781,1200
14607,2019-12-30,0,-430.399784,2.632365e+18,2019.994521,1200
14608,2019-12-31,0,-432.312062,2.632365e+18,2019.997260,1200


In [ ]:
Y_true.to_parquet("tests/data/test_sensitivity.parquet")

In [ ]:
ds = xr.open_mfdataset("tests/data/ts_gris_g1200m_v2023_RAGIS_id_*.nc", combine="nested", concat_dim="id", parallel=True)

In [ ]:
ens_vars = ["grounding_line_flux", "limnsw"]
ens = ds.sel(time=slice("1980-01-01" ,"1983-01-01"))[ens_vars].resample(time="1AS").mean()
ens_df = ens.to_dataframe().reset_index().dropna()

In [ ]:
ens_vars

In [ ]:
n_jobs=4

Y_true_xr = sensitivity_analysis(ens_df, ensemble_file=ensemble_file, n_jobs=n_jobs, calc_variables=ens_vars)[sens_vars]

In [10]:
    df_parquet_true = pd.read_parquet("../tests/data/test_scalar_file_YM.parquet")
    df_csv_true = pd.read_csv(
        "../tests/data/test_scalar_file_YM.csv",
        index_col=0,
        infer_datetime_format=True,
        parse_dates=["time"],
    )

    df = ncfile2dataframe(infile, resample="yearly", add_vars=False)
    assert_frame_equal(df, df_parquet_true)
    assert_frame_equal(df, df_csv_true)


In [91]:
def ncfile2dataframe(
    infile: Union[str, pathlib.Path],
    resample: Union[str, None] = None,
    add_vars: bool = True,
    norm_year: Union[None, float] = None,
    verbose: bool = False,
) -> pd.DataFrame:
    """Convert netCDF file to pandas.DataFrame"""

    if isinstance(infile, pathlib.Path):
        assert infile.exists()
    else:
        assert os.path.isfile(infile)
    if verbose:
        print(f"Opening {infile}")
    with xr.open_dataset(infile) as ds:
        if resample == "monthly":
            ds = ds.resample(time="1MS").mean()
        elif resample == "yearly":
            ds = ds.resample(time="1YS").mean()
        else:
            pass
        if isinstance(infile, pathlib.Path):
            m_id_re = re.search("id_(.+?)_", str(infile))
        else:
            m_id_re = re.search("id_(.+?)_", infile)
        assert m_id_re is not None
        m_id: Union[str, int]
        try:
            m_id = int(m_id_re.group(1))
        except:
            m_id = str(m_id_re.group(1))

        if isinstance(infile, pathlib.Path):
            m_dx_re = re.search("gris_g(.+?)m", str(infile))
        else:
            m_dx_re = re.search("gris_g(.+?)m", infile)
        assert m_dx_re is not None
        m_dx = int(m_dx_re.group(1))
        datetimeindex = ds.indexes["time"]
        years = [to_decimal_year(x.to_pydatetime()) for x in datetimeindex]
        nt = len(datetimeindex)
        id_S = pd.Series(data=np.repeat(m_id, nt), index=datetimeindex, name="id")
        S = [id_S]
        for m_var in ds.data_vars:
            if m_var not in (
                "time_bounds",
                "time_bnds",
                "timestamp",
                "run_stats",
                "pism_config",
            ):
                if hasattr(ds[m_var], "units"):
                    m_units = ds[m_var].units
                    m_S_name = f"{m_var} ({m_units})"
                else:
                    m_units = ""
                    m_S_name = f"{m_var}"
                data = np.squeeze(ds[m_var].values)
                m_S = pd.Series(data=data, index=datetimeindex, name=m_S_name)
                S.append(m_S)
        m_Y = pd.Series(data=years, index=datetimeindex, name="Year")
        S.append(m_Y)
        df = pd.concat(S, axis=1).reset_index()
        df["resolution_m"] = m_dx
        
        if add_vars:
            df = add_vars_to_dataframe(df)
        
        if norm_year:
            norm_year_idx = np.nonzero(np.array(years) == norm_year)[0][0]
            df["limnsw (kg)"] -= df["limnsw (kg)"][norm_year_idx]
            if add_vars:
                df["Cumulative ice sheet mass change (Gt)"] += df[
                    "Cumulative ice sheet mass change (Gt)"
                ][norm_year_idx]
                df["SLE (cm)"] += df["SLE (cm)"][norm_year_idx]

    return df


def add_vars_to_dataframe(df: pd.DataFrame):
    """Add additional variables to DataFrame"""

    if "limnsw (kg)" in df.columns:
        df["Cumulative ice sheet mass change (Gt)"] = (
            df["limnsw (kg)"] - df["limnsw (kg)"][0]
        ) / 1e12
        df["SLE (cm)"] = df["Cumulative ice sheet mass change (Gt)"] * gt2cmsle
        if "grounding_line_flux (Gt year-1)" in df.columns:
            df["Rate of ice discharge (Gt/yr)"] = -df["grounding_line_flux (Gt year-1)"]
        if "tendency_of_ice_mass_due_to_surface_mass_flux (Gt year-1)" in df.columns:
            df["Rate of surface mass balance (Gt/yr)"] = df[
                "tendency_of_ice_mass_due_to_surface_mass_flux (Gt year-1)"
            ]
    return df


In [82]:
    df_parquet_true = pd.read_parquet("../tests/data/test_scalar_file.parquet")
    df_csv_true = pd.read_csv(
        "../tests/data/test_scalar_file.csv",
        index_col=0,
        infer_datetime_format=True,
        parse_dates=["time"],
    )

    df = ncfile2dataframe(infile, add_vars=False)
    assert_frame_equal(df, df_parquet_true)
    assert_frame_equal(df, df_csv_true)

    df_parquet_true = pd.read_parquet("../tests/data/test_scalar_file_YM.parquet")
    df_csv_true = pd.read_csv(
        "../tests/data/test_scalar_file_YM.csv",
        index_col=0,
        infer_datetime_format=True,
        parse_dates=["time"],
    )

    df = ncfile2dataframe(infile, resample="yearly", add_vars=False)
    assert_frame_equal(df, df_parquet_true)
    assert_frame_equal(df, df_csv_true)


In [70]:
infile

'../tests/data/ts_gris_g1200m_v2023_RAGIS_id_0_1980-1-1_2020-1-1.nc'

In [86]:
    df_parquet_true = pd.read_parquet("../tests/data/test_scalar_file_add_vars.parquet")


In [87]:
df_parquet_true

,time,id,grounding_line_flux (Gt year-1),limnsw (kg),Year,resolution_m,Cumulative ice sheet mass change (Gt),SLE (cm),Rate of ice discharge (Gt/yr)
0,1980-01-02,0,-549.268836,2.633908e+18,1980.002732,1200,35.801685,-0.009876,-549.268836
1,1980-01-03,0,-531.800248,2.633909e+18,1980.005464,1200,36.401539,-0.010042,-531.800248
2,1980-01-04,0,-524.580911,2.633910e+18,1980.008197,1200,37.138779,-0.010245,-524.580911
3,1980-01-05,0,-542.280141,2.633911e+18,1980.010929,1200,38.330877,-0.010574,-542.280141
4,1980-01-06,0,-582.518599,2.633911e+18,1980.013661,1200,38.351885,-0.010580,-582.518599
...,...,...,...,...,...,...,...,...,...
14605,2019-12-28,0,-434.405629,2.632364e+18,2019.989041,1200,-1508.145279,0.416040,-434.405629
14606,2019-12-29,0,-430.935017,2.632365e+18,2019.991781,1200,-1507.752121,0.415932,-430.935017
14607,2019-12-30,0,-430.399784,2.632365e+18,2019.994521,1200,-1507.714305,0.415921,-430.399784
14608,2019-12-31,0,-432.312062,2.632365e+18,2019.997260,1200,-1507.691166,0.415915,-432.312062


In [96]:
    df = ncfile2dataframe(infile, add_vars=True, norm_year=1992.0)


In [97]:
df

,time,id,grounding_line_flux (Gt year-1),limnsw (kg),Year,resolution_m,Cumulative ice sheet mass change (Gt),SLE (cm),Rate of ice discharge (Gt/yr)
0,1980-01-02,0,-549.268836,3.580168e+13,1980.002732,1200,-35.801685,-0.009876,549.268836
1,1980-01-03,0,-531.800248,3.640154e+13,1980.005464,1200,-35.201831,-0.009711,531.800248
2,1980-01-04,0,-524.580911,3.713878e+13,1980.008197,1200,-34.464590,-0.009507,524.580911
3,1980-01-05,0,-542.280141,3.833088e+13,1980.010929,1200,-33.272493,-0.009179,542.280141
4,1980-01-06,0,-582.518599,3.835189e+13,1980.013661,1200,-33.251484,-0.009173,582.518599
...,...,...,...,...,...,...,...,...,...
14605,2019-12-28,0,-434.405629,-1.508145e+15,2019.989041,1200,-1579.748649,-0.435793,434.405629
14606,2019-12-29,0,-430.935017,-1.507752e+15,2019.991781,1200,-1579.355490,-0.435684,430.935017
14607,2019-12-30,0,-430.399784,-1.507714e+15,2019.994521,1200,-1579.317674,-0.435674,430.399784
14608,2019-12-31,0,-432.312062,-1.507691e+15,2019.997260,1200,-1579.294535,-0.435667,432.312062


In [85]:
df_parquet_true

,time,id,grounding_line_flux (Gt year-1),limnsw (kg),Year,resolution_m
0,1980-01-01,0,-495.444694,2.633911e+18,1980.0,1200
1,1981-01-01,0,-471.423994,2.633839e+18,1981.0,1200
2,1982-01-01,0,-468.614588,2.633765e+18,1982.0,1200
3,1983-01-01,0,-473.670005,2.633774e+18,1983.0,1200
4,1984-01-01,0,-475.791586,2.633848e+18,1984.0,1200
5,1985-01-01,0,-465.904673,2.633901e+18,1985.0,1200
6,1986-01-01,0,-468.651741,2.633931e+18,1986.0,1200
7,1987-01-01,0,-482.000942,2.634015e+18,1987.0,1200
8,1988-01-01,0,-459.679830,2.633991e+18,1988.0,1200
9,1989-01-01,0,-466.374171,2.633906e+18,1989.0,1200


In [98]:
def convert_netcdf_to_dataframe(
    infiles: list,
    resample: Union[str, None] = None,
    n_jobs: int = 4,
    add_vars: bool = True,
    norm_year: Union[None, float] = None,
    verbose: bool = False,
) -> pd.DataFrame:
    """
    Convert list of netCDF files to Pandas DataFrame.


    """
    n_files = len(infiles)
    print("Converting netcdf files to pandas.DataFrame")
    print("-------------------------------------------")
    start_time = time.perf_counter()
    with tqdm_joblib(tqdm(desc="Processing files", total=n_files)) as progress_bar:
        result = Parallel(n_jobs=n_jobs)(
            delayed(ncfile2dataframe)(infile, resample, add_vars, norm_year, verbose)
            for infile in infiles
        )
        del progress_bar
    finish_time = time.perf_counter()
    time_elapsed = finish_time - start_time
    print(f"Conversion finished in {time_elapsed:.0f} seconds")
    print("-------------------------------------------")

    df = pd.concat(result)

    return df.sort_values(by=["time", "id"]).reset_index(drop=True)


In [104]:
    df_parquet_true = pd.read_parquet("../tests/data/test_scalar_add_vars.parquet")
    infiles = glob("../tests/data/ts_gris_g1200m_v2023_RAGIS_id_*_1980-1-1_2020-1-1.nc")

    df = convert_netcdf_to_dataframe(infiles, add_vars=True, norm_year=1992.0)


Converting netcdf files to pandas.DataFrame
-------------------------------------------


Processing files: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 35.46it/s]


Conversion finished in 1 seconds
-------------------------------------------


In [101]:
@contextlib.contextmanager
def tqdm_joblib(tqdm_object):
    """Context manager to patch joblib to report into tqdm progress bar given as argument"""

    class TqdmBatchCompletionCallback(joblib.parallel.BatchCompletionCallBack):
        """TQDM Callback"""

        def __call__(self, *args, **kwargs):
            tqdm_object.update(n=self.batch_size)
            return super().__call__(*args, **kwargs)

    old_batch_callback = joblib.parallel.BatchCompletionCallBack
    joblib.parallel.BatchCompletionCallBack = TqdmBatchCompletionCallback
    try:
        yield tqdm_object
    finally:
        joblib.parallel.BatchCompletionCallBack = old_batch_callback
        tqdm_object.close()


In [105]:
df

,time,id,grounding_line_flux (Gt year-1),limnsw (kg),Year,resolution_m,Cumulative ice sheet mass change (Gt),SLE (cm),Rate of ice discharge (Gt/yr)
0,1980-01-02,0,-549.268836,3.580168e+13,1980.002732,1200,-35.801685,-0.009876,549.268836
1,1980-01-02,1,-683.276005,8.380394e+14,1980.002732,1200,-838.039377,-0.231183,683.276005
2,1980-01-02,2,-324.823461,-3.556732e+13,1980.002732,1200,35.567317,0.009812,324.823461
3,1980-01-02,3,-424.347672,-8.479677e+14,1980.002732,1200,847.967731,0.233922,424.347672
4,1980-01-02,4,-347.801638,-6.431086e+14,1980.002732,1200,643.108590,0.177409,347.801638
...,...,...,...,...,...,...,...,...,...
341412,2020-01-01,20,-415.598540,-5.773333e+13,2020.000000,1200,1425.024450,0.393110,415.598540
341413,2020-01-01,21,-366.514897,1.112349e+15,2020.000000,1200,3869.174400,1.067358,366.514897
341414,2020-01-01,22,-465.439380,-2.395211e+15,2020.000000,1200,-4133.858622,-1.140375,465.439380
341415,2020-01-01,23,-488.749311,-1.884567e+15,2020.000000,1200,-2761.250962,-0.761724,488.749311


In [106]:
    infiles = glob("../tests/data/ts_gris_g1200m_v2023_RAGIS_id_*_1980-1-1_1982-1-1.nc")
    df = convert_netcdf_to_dataframe(infiles, resample="yearly", add_vars=False)


Converting netcdf files to pandas.DataFrame
-------------------------------------------


Processing files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 154.51it/s]

Conversion finished in 0 seconds
-------------------------------------------
